In [1]:
import requests
import json
import pandas as pd
import time
from tqdm import tqdm
from pathlib import Path

In [2]:

DATA_ROOT = Path(r"C:\Users\shawn\PycharmProjects\NetworkFT\networkft\data")
folders = ["raw", "intermediate", "primary"]
DATA_PATH = {
    folder: DATA_ROOT.joinpath(f"0{i+1}_{folder}")
    for i, folder in enumerate(folders)
}

In [ ]:
CONTRACTS = {
    "azuki": "0xED5AF388653567Af2F388E6224dC7C4b3241C544",
    "doodles": "0x8a90CAb2b38dba80c64b7734e58Ee1dB38B8992e",
    "moonbirds": "0x23581767a106ae21c074b2276D25e5C3e136a68b",
    "clonex": "0x348fc118bcc65a92dc033a951af153d14d945312",
    "mayc": "0x60E4d786628Fea6478F785A6d7e704777c86a7c6",
    "pxn": "0x160C404B2b49CBC3240055CEaEE026df1e8497A0",
    "goblintown": "0xbCe3781ae7Ca1a5e050Bd9C4c77369867eBc307e"
}

def generate_endpoint(
    topic, 
    contract_address, 
    category="tokens",
    chain_id=1,
    base_url=None
):
    """Generates the endpoint """
    base_url = base_url or "https://api.covalenthq.com/v1"
    
    endpoint = f"{chain_id}/{category}/{contract_address}/{topic}"
    url = f"{base_url}/{endpoint}/"
    return url

def get_covalent_query(url, params={}):
    """Queries the covalent API given the url endpoint and extra params"""
    params = params or {}
    params.update({"key": KEY})
    r = requests.get(url=url, params=params)
    result = r.json()
    if result["error"]:
        print(result["error_message"])
        return None
    else:
        return result["data"]

def save_json(json_file, path):
    """Saves a dict as a json in the path"""
    with open(path, "w") as f:
        json.dump(json_file, f)
        
def load_json(path):
    """Loads json as dict"""
    with open(path, "rb") as f:
        file = json.load(f)
    return file
    
def check_directory(path):
    """Checks if directory exists, empty contents if yes, creates if no"""
    if not path.is_dir():
        path.mkdir(parents=True, exist_ok=False)
    else:
        for content in path.iterdir():
            content.unlink()

def update_saved_file(path, json):
    """Updates the json found in the path with the provided json."""
    if json:
        txs = load_json(path)
        txs.update(json)
        save_json(txs, path)

In [ ]:
def parse_token_ids(result):
    """Parses covalent response into a list of ids"""
    token_ids = [item["token_id"] for item in result["items"]]
    return token_ids

In [ ]:
def parse_token_tx(result):
    """Parses covalent response to token transactions. Take the first dict in log_events
    because that is the most important one. Subsequent ones are signatures, etc.
    """
    txs = [tx["log_events"][0] for tx in result["items"][0]["nft_transactions"]]
    
    parsed = {
        "updated_at": result["updated_at"],
        "txs": txs
    }
    return parsed

In [11]:
def query_nft_data(nft, contract_address, token_ids=True, txs=True, save=None, resume=True):
    """Queries covalent API for all token id and transaction data
    of an NFT collection, and optionally saves it as a pkl
    
    Args
        nft: nft project name
        token_ids: list of token ids
        txs: pull transaction data for all token ids
        save: path to save file
        resume: continue from last saved file
    
    Returns
        Dict of ids and txs 
    
    """
    # Empty directory of existing transactions
    if save and not resume: 
        check_directory(save)
        
    # Load downloaded transactions if possible
    txs_fp = save.joinpath(f"{nft}_txs.json")
    if resume and txs_fp.exists():
        nft_data = load_json(txs_fp)
        saved_id = set(list(nft_data))
    else:
        nft_data = {}
        saved_id = set([])

    # First get all the ids
    id_url = generate_endpoint("nft_token_ids", contract_address)
    result_id = get_covalent_query(id_url)
    if save:
        save_json(result_id, save.joinpath(f"{nft}_ids.json"))
    ids = parse_token_ids(result_id)
    print(f"Found {len(ids)} for token {nft}. Finding transactions...")
    
    # Get transactions for each token id
    counter = 0
    for _id in tqdm(ids, f"downloading transactions for {nft}"):
        
        if _id in saved_id:
            continue
        # Save after interval
        if counter >= 500:
            if txs_fp.exists():
                update_saved_file(txs_fp, nft_data)
                print(f"udpating save file to id {_id}")
            else:
                save_json(nft_data, save.joinpath(f"{nft}_txs.json"))
                print(f"Saving data up to id {_id}")
            counter = 0
            # Remembers which id has been saved and clear the data variable
            saved_id.update(list(nft_data))
            nft_data = {}
            
        try:
            #time.sleep(0.1)
            token_url = generate_endpoint(f"nft_transactions/{_id}", contract_address)
            result = get_covalent_query(token_url)
            result_parsed = parse_token_tx(result)
            nft_data[_id] = result_parsed
            counter += 1
        except Exception as e:
            print(e)
            continue

    update_saved_file(txs_fp, nft_data)
    return nft_data
        

In [ ]:
nft_data = {}
for nft, contract_address in CONTRACTS.items():
    nft_data[nft] = query_nft_data(nft, contract_address, save=DATA_PATH["raw"])

Found 10000 for token azuki. Finding transactions...


downloading transactions for azuki:   2%|████▏                                                                                                                                                                                                               | 200/10000 [02:30<2:24:24,  1.13it/s]

Saving data up to id 200


downloading transactions for azuki:   4%|████████▍                                                                                                                                                                                                           | 400/10000 [05:31<2:20:42,  1.14it/s]

udpating save file to id 400


...

downloading transactions for mayc:  85%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                              | 16601/19418 [6:47:43<1:08:17,  1.45s/it]

udpating save file to id 23930


downloading transactions for mayc:  87%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                            | 16801/19418 [6:52:37<1:04:31,  1.48s/it]

udpating save file to id 24338


downloading transactions for mayc:  87%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                           | 16887/19418 [6:54:49<1:01:22,  1.46s/it]

In [ ]:
txs = load_json(DATA_PATH["raw"].joinpath("azuki_txs.json"))

In [55]:
token_url = generate_endpoint(f"nft_transactions/9911", CONTRACTS["azuki"])
result = get_covalent_query(token_url)

In [48]:
logs = [event["decoded"] for event in txs['10']["items"][0]["nft_transactions"][0]["log_events"]]

In [54]:
len(check)

200